In [ ]:
NEO4J_URI=""
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=""

In [ ]:
!pip install langchain-community
from langchain_community.graphs import Neo4jGraph
!pip install neo4j

NEO4J_URI=""
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=""

graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD,refresh_schema=False)

graph

In [30]:
### MOVIE Dataset
### MOVIE Dataset

movie_query = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies_small.csv" as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m)) // Added the missing hyphen here
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)) // Added the missing hyphen here
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)) // Added the missing hyphen here

"""




In [31]:
movie_query

'\nLOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies_small.csv" as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, \'|\') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m)) // Added the missing hyphen here\nFOREACH (actor in split(row.actors, \'|\') |\n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m)) // Added the missing hyphen here\nFOREACH (genre in split(row.genres, \'|\') |\n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g)) // Added the missing hyphen here\n\n'

In [32]:
graph.query(movie_query)

[]

In [ ]:
groq_api = ""

In [34]:

%pip uninstall ChatGroq -y

%pip install langchain-groq


from langchain_groq import ChatGroq


llm =ChatGroq(api_key=groq_api,model_name = "Gemma2-9b-It", verbose=True)

In [35]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq

# Ensure the graph schema is refreshed
graph.refresh_schema()

# Re-initialize the chain with the refreshed graph schema
chain = GraphCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

# Now try the query again
response = chain.invoke({"query":"who was the director of the movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'who was the director of the movie Casino',
 'result': 'Martin Scorsese  \n'}

In [40]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq

# Ensure the graph schema is refreshed
graph.refresh_schema()

# Re-initialize the chain with the refreshed graph schema
chain = GraphCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

# Now try the query again
response = chain.invoke({"query":"How many movies has Tom Hanks acted in?"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN count(m)
Full Context:
[{'count(m)': 8}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer. \n"}

## Prompt Strategies

In [50]:
chain = GraphCypherQAChain.from_llm(graph=graph,llm=llm,exclude_types=['Genre'],verbose=True,allow_dangerous_requests=True)
chain



GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7d0a244b9f50>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d09dca9c850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d09dcac51d0>, model_name='Gemma2-9b-I

In [51]:
chain.graph_schema

'Node properties are the following:\nCEO {POB: STRING, name: STRING, YOB: INTEGER},company {name: STRING},entrepreneur {POB: STRING, name: STRING, YOB: INTEGER},Country {name: STRING},Person {name: STRING, born: INTEGER},Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT},User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER},post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:DIRECTED]->(:Movie),(:User)-[:posted]->(:post),(:User)-[:LIKES]->(:User),(:User)-[:FRIEND]->(:User)'

In [49]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name", # Changed from {{title: 'Casino'}}
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {title: 'Schindler\\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name", # Changed from {{title: 'Schindler\'s List'}}
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1", # Changed from (a:Actor) to (a:Person) assuming 'Actor' node is not used
    },
]



In [70]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)

examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<--[:ACTED_IN]--(a) RETURN a.name", # Escaped curly braces
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)", # Escaped curly braces
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name", # Escaped curly braces
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]


prompt = FewShotPromptTemplate(
    examples=examples[:8],
    example_prompt=example_prompt,
    prefix = "you are a Neo4j expert. Given an input question, create a very accurate question",
    input_variables=['question','schema'],
    suffix="User input: {question}\nCypher query:",
)

In [71]:
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<--[:ACTED_IN]--(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}, {'question': "Which directors have made 

In [72]:
print(prompt.format(question="How many movies has Tom Hanks acted in?",schema=chain.graph_schema))

you are a Neo4j expert. Given an input question, create a very accurate question

User input: How many artists are there?
Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input: Which actors played in the movie Casino?
Cypher query: MATCH (m:Movie {title: 'Casino'})<--[:ACTED_IN]--(a) RETURN a.name

User input: How many movies has Tom Hanks acted in?
Cypher query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

User input: List all the genres of the movie Schindler's List
Cypher query: MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

User input: Which actors have worked in movies from both the comedy and action genres?
Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input: Which directors have made movies with at least three different 

In [73]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d09dca9c850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d09dcac51d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [74]:
chain = GraphCypherQAChain.from_llm(graph=graph,llm=llm,cypher_prompt=prompt,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7d0a244b9f50>, cypher_generation_chain=LLMChain(verbose=False, prompt=FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<--[:ACTED_IN]--(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:

In [75]:
chain.invoke("Which actors played in the movie Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name 




Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods played in the movie Casino. \n'}

In [76]:
chain.invoke("How many movies has Tom Hanks acted in?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m) 

Full Context:
[{'count(m)': 8}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer. \n"}

In [78]:
chain.invoke("list all the genres of the movie Casino")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})-[:IN_GENRE]->(g:Genre) RETURN g.name 




Full Context:
[{'g.name': 'Drama'}, {'g.name': 'Crime'}]

> Finished chain.


{'query': 'list all the genres of the movie Casino',
 'result': 'Drama, Crime \n'}